<a href="https://colab.research.google.com/github/Tariik7/BIGDATALABS/blob/main/LAB_SPARK_SQL_FIFA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q findspark pyspark py4j
!pip install -q findspark
!pip install pyspark
!pip install py4j
import pyspark
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder \
    .appName("FIFA World Cup Analysis") \
    .getOrCreate()
print(" Spark est configuré avec succès !")

 Spark est configuré avec succès !


In [5]:
df=spark.read.csv('/content/fifaworldcup.csv',inferSchema=True,header=True)
df.printSchema()


root
 |-- date: date (nullable = true)
 |-- home_team: string (nullable = true)
 |-- away_team: string (nullable = true)
 |-- home_score: string (nullable = true)
 |-- away_score: string (nullable = true)
 |-- tournament: string (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- neutral: boolean (nullable = true)



In [6]:
df.createOrReplaceTempView("football")

**Découverte & requêtes simples**

In [7]:
spark.sql("SELECT count(*) AS total_matchs FROM football").show()

+------------+
|total_matchs|
+------------+
|       44152|
+------------+



In [8]:
spark.sql("SELECT min(year(date)) as debut, max(year(date)) as fin FROM football").show()

+-----+----+
|debut| fin|
+-----+----+
| 1872|2022|
+-----+----+



In [11]:
df.show(1)

+----------+---------+---------+----------+----------+----------+-------+--------+-------+
|      date|home_team|away_team|home_score|away_score|tournament|   city| country|neutral|
+----------+---------+---------+----------+----------+----------+-------+--------+-------+
|1872-11-30| Scotland|  England|         0|         0|  Friendly|Glasgow|Scotland|  false|
+----------+---------+---------+----------+----------+----------+-------+--------+-------+
only showing top 1 row


In [13]:
spark.sql("SELECT tournament, count(*) as n FROM football GROUP BY tournament ORDER BY n DESC LIMIT 10").show()

+--------------------+-----+
|          tournament|    n|
+--------------------+-----+
|            Friendly|17461|
|FIFA World Cup qu...| 7774|
|UEFA Euro qualifi...| 2593|
|African Cup of Na...| 1932|
|      FIFA World Cup|  948|
|        Copa América|  841|
|AFC Asian Cup qua...|  764|
|African Cup of Na...|  742|
|          CECAFA Cup|  620|
|CFU Caribbean Cup...|  606|
+--------------------+-----+



In [15]:
spark.sql("SELECT count(*) FROM football WHERE home_score = away_score").show()

+--------+
|count(1)|
+--------+
|   10213|
+--------+



In [28]:
spark.sql("SELECT * FROM football WHERE (CAST(home_score AS INT) + CAST(away_score AS INT)) > 6").show()

+----------+----------------+----------------+----------+----------+--------------------+---------+--------+-------+
|      date|       home_team|       away_team|home_score|away_score|          tournament|     city| country|neutral|
+----------+----------------+----------------+----------+----------+--------------------+---------+--------+-------+
|1878-03-02|        Scotland|         England|         7|         2|            Friendly|  Glasgow|Scotland|  false|
|1878-03-23|        Scotland|           Wales|         9|         0|            Friendly|  Glasgow|Scotland|  false|
|1879-04-05|         England|        Scotland|         5|         4|            Friendly|   London| England|  false|
|1880-03-13|        Scotland|         England|         5|         4|            Friendly|  Glasgow|Scotland|  false|
|1881-03-12|         England|        Scotland|         1|         6|            Friendly|   London| England|  false|
|1882-02-18|Northern Ireland|         England|         0|       

**Agrégations & statistiques**

In [29]:
spark.sql("SELECT equipe, count(*) as total FROM ( SELECT home_team AS equipe FROM football UNION ALL SELECT away_team AS equipe FROM football ) GROUP BY equipe ORDER BY total DESC;").show()

+-----------+-----+
|     equipe|total|
+-----------+-----+
|     Sweden| 1052|
|    England| 1047|
|     Brazil| 1019|
|  Argentina| 1014|
|    Germany|  986|
|    Hungary|  964|
|     Mexico|  926|
|    Uruguay|  919|
|South Korea|  904|
|     France|  875|
|     Poland|  851|
|      Italy|  837|
|Switzerland|  834|
|     Norway|  832|
|    Denmark|  832|
|    Austria|  820|
|Netherlands|  820|
|   Scotland|  814|
|    Belgium|  804|
|      Chile|  793|
+-----------+-----+
only showing top 20 rows


In [31]:
spark.sql("""
SELECT equipe, sum(buts) as total_buts FROM (
    SELECT home_team AS equipe, COALESCE(TRY_CAST(home_score AS INT), 0) as buts FROM football
    UNION ALL
    SELECT away_team AS equipe, COALESCE(TRY_CAST(away_score AS INT), 0) as buts FROM football
) GROUP BY equipe ORDER BY total_buts DESC LIMIT 10
""").show()

+-----------+----------+
|     equipe|total_buts|
+-----------+----------+
|    England|      2282|
|     Brazil|      2228|
|    Germany|      2205|
|     Sweden|      2060|
|    Hungary|      1948|
|  Argentina|      1880|
|Netherlands|      1694|
|     Mexico|      1592|
|South Korea|      1576|
|     France|      1560|
+-----------+----------+



In [32]:
spark.sql("""
SELECT floor(year(date)/10)*10 AS decade,
       avg(COALESCE(TRY_CAST(home_score AS INT), 0) + COALESCE(TRY_CAST(away_score AS INT), 0)) AS moyenne
FROM football
GROUP BY decade ORDER BY decade
""").show()

+------+------------------+
|decade|           moyenne|
+------+------------------+
|  1870| 4.538461538461538|
|  1880| 5.581818181818182|
|  1890|5.1525423728813555|
|  1900| 4.169354838709677|
|  1910| 4.007067137809187|
|  1920|3.8367626886145403|
|  1930|  4.25940594059406|
|  1940| 4.294776119402985|
|  1950| 3.980964467005076|
|  1960|3.4683274021352313|
|  1970| 2.955379908210097|
|  1980|2.4950990615224193|
|  1990| 2.730442692540934|
|  2000| 2.792825302483549|
|  2010|  2.69994863893169|
|  2020| 2.544389364578639|
+------+------------------+



In [33]:
spark.sql("""
    SELECT tournament, year(date) as annee, count(*) as nb_matchs
    FROM football
    GROUP BY tournament, annee
    ORDER BY annee DESC, nb_matchs DESC
""").show()

+--------------------+-----+---------+
|          tournament|annee|nb_matchs|
+--------------------+-----+---------+
|            Friendly| 2022|      281|
| UEFA Nations League| 2022|      160|
|FIFA World Cup qu...| 2022|       99|
|CONCACAF Nations ...| 2022|       67|
|African Cup of Na...| 2022|       54|
|African Cup of Na...| 2022|       52|
|      FIFA World Cup| 2022|       48|
|AFC Asian Cup qua...| 2022|       36|
|MSG Prime Ministe...| 2022|        6|
|          Navruz Cup| 2022|        4|
|Jordan Internatio...| 2022|        4|
|          King's Cup| 2022|        4|
|           Kirin Cup| 2022|        4|
|CONIFA Africa Foo...| 2022|        4|
|CONIFA South Amer...| 2022|        3|
|AFF Championship ...| 2022|        2|
|          Baltic Cup| 2022|        2|
| Kirin Challenge Cup| 2022|        2|
|CONMEBOL–UEFA Cup...| 2022|        1|
|FIFA World Cup qu...| 2021|      607|
+--------------------+-----+---------+
only showing top 20 rows


In [34]:
spark.sql("""
    SELECT home_team, count(*) as victoires_domicile
    FROM football
    WHERE COALESCE(TRY_CAST(home_score AS INT), 0) > COALESCE(TRY_CAST(away_score AS INT), 0)
    GROUP BY home_team
    ORDER BY victoires_domicile DESC
""").show(10)

+-----------+------------------+
|  home_team|victoires_domicile|
+-----------+------------------+
|     Brazil|               423|
|  Argentina|               373|
|    Germany|               327|
|    England|               324|
|     Mexico|               320|
|South Korea|               296|
|     Sweden|               295|
|      Italy|               291|
|     France|               290|
|    Hungary|               265|
+-----------+------------------+
only showing top 10 rows


In [35]:
spark.sql("""
    SELECT equipe,
           SUM(CASE WHEN resultat = 'V' THEN 1 ELSE 0 END) as victoires,
           SUM(CASE WHEN resultat = 'D' THEN 1 ELSE 0 END) as defaites,
           SUM(CASE WHEN resultat = 'N' THEN 1 ELSE 0 END) as nuls
    FROM (
        SELECT home_team as equipe,
               CASE WHEN COALESCE(TRY_CAST(home_score AS INT), 0) > COALESCE(TRY_CAST(away_score AS INT), 0) THEN 'V'
                    WHEN COALESCE(TRY_CAST(home_score AS INT), 0) < COALESCE(TRY_CAST(away_score AS INT), 0) THEN 'D'
                    ELSE 'N' END as resultat FROM football
        UNION ALL
        SELECT away_team as equipe,
               CASE WHEN COALESCE(TRY_CAST(away_score AS INT), 0) > COALESCE(TRY_CAST(home_score AS INT), 0) THEN 'V'
                    WHEN COALESCE(TRY_CAST(away_score AS INT), 0) < COALESCE(TRY_CAST(home_score AS INT), 0) THEN 'D'
                    ELSE 'N' END as resultat FROM football
    ) GROUP BY equipe ORDER BY victoires DESC
""").show(10)

+-----------+---------+--------+----+
|     equipe|victoires|defaites|nuls|
+-----------+---------+--------+----+
|     Brazil|      651|     160| 208|
|    England|      594|     200| 253|
|    Germany|      573|     205| 208|
|  Argentina|      547|     213| 254|
|     Sweden|      517|     308| 227|
|South Korea|      477|     192| 235|
|     Mexico|      467|     243| 216|
|    Hungary|      452|     301| 211|
|      Italy|      445|     157| 235|
|     France|      436|     249| 190|
+-----------+---------+--------+----+
only showing top 10 rows


In [37]:
spark.sql("""
    SELECT neutral, AVG(COALESCE(TRY_CAST(home_score AS INT), 0) + COALESCE(TRY_CAST(away_score AS INT), 0)) as moyenne_buts
    FROM football
    GROUP BY neutral
""").show()

+-------+------------------+
|neutral|      moyenne_buts|
+-------+------------------+
|   true|3.0027282648235722|
|  false|2.8858728435275665|
+-------+------------------+



In [38]:
spark.sql("""
    SELECT date, home_team, away_team, home_score, away_score,
           ABS(COALESCE(TRY_CAST(home_score AS INT), 0) - COALESCE(TRY_CAST(away_score AS INT), 0)) as ecart
    FROM football
    ORDER BY ecart DESC LIMIT 5
""").show()

+----------+---------+--------------+----------+----------+-----+
|      date|home_team|     away_team|home_score|away_score|ecart|
+----------+---------+--------------+----------+----------+-----+
|2001-04-11|Australia|American Samoa|        31|         0|   31|
|1971-09-13|   Tahiti|  Cook Islands|        30|         0|   30|
|1979-08-30|     Fiji|      Kiribati|        24|         0|   24|
|2001-04-09|Australia|         Tonga|        22|         0|   22|
|1966-04-03|    Libya|          Oman|        21|         0|   21|
+----------+---------+--------------+----------+----------+-----+



In [40]:
spark.sql("""
    SELECT equipe, (SUM(buts_marques) - SUM(buts_encaisses)) as goal_average
    FROM (
        SELECT home_team as equipe, COALESCE(TRY_CAST(home_score AS INT), 0) as buts_marques, COALESCE(TRY_CAST(away_score AS INT), 0) as buts_encaisses FROM football
        UNION ALL
        SELECT away_team as equipe, COALESCE(TRY_CAST(away_score AS INT), 0) as buts_marques, COALESCE(TRY_CAST(home_score AS INT), 0) as buts_encaisses FROM football
    ) GROUP BY equipe ORDER BY goal_average DESC
""").show(10)

+-----------+------------+
|     equipe|goal_average|
+-----------+------------+
|     Brazil|        1318|
|    England|        1268|
|    Germany|        1072|
|  Argentina|         838|
|      Spain|         814|
|South Korea|         768|
|     Sweden|         713|
|Netherlands|         694|
|      Italy|         638|
|     Mexico|         616|
+-----------+------------+
only showing top 10 rows


**Requêtes analytiques (fenêtres & logique avancée)**

In [41]:
spark.sql("""
WITH victoires_annuelles AS (
    SELECT year(date) as annee, vainqueur, count(*) as nb_victoires
    FROM (
        SELECT date,
               CASE WHEN COALESCE(TRY_CAST(home_score AS INT), 0) > COALESCE(TRY_CAST(away_score AS INT), 0) THEN home_team
                    WHEN COALESCE(TRY_CAST(away_score AS INT), 0) > COALESCE(TRY_CAST(home_score AS INT), 0) THEN away_team
               END as vainqueur
        FROM football
    )
    WHERE vainqueur IS NOT NULL
    GROUP BY annee, vainqueur
)
SELECT annee, vainqueur, nb_victoires,
       ROW_NUMBER() OVER(PARTITION BY annee ORDER BY nb_victoires DESC) as rang
FROM victoires_annuelles
ORDER BY annee DESC, rang ASC
""").show()

+-----+------------+------------+----+
|annee|   vainqueur|nb_victoires|rang|
+-----+------------+------------+----+
| 2022| South Korea|           9|   1|
| 2022|  Costa Rica|           9|   2|
| 2022|     Bahrain|           9|   3|
| 2022|  Uzbekistan|           8|   4|
| 2022|   Argentina|           8|   5|
| 2022|     Senegal|           8|   6|
| 2022|     Algeria|           8|   7|
| 2022|     Tunisia|           8|   8|
| 2022|      Mexico|           8|   9|
| 2022|     Morocco|           7|  10|
| 2022|Burkina Faso|           7|  11|
| 2022|      Brazil|           7|  12|
| 2022|     Uruguay|           7|  13|
| 2022|       Egypt|           7|  14|
| 2022|    Thailand|           7|  15|
| 2022|      Jordan|           7|  16|
| 2022|    Cameroon|           6|  17|
| 2022|      Canada|           6|  18|
| 2022|   Indonesia|           6|  19|
| 2022|      Greece|           6|  20|
+-----+------------+------------+----+
only showing top 20 rows


In [42]:
spark.sql("""
SELECT floor(year(date)/10)*10 as decennie, count(*) as total_matchs
FROM football
GROUP BY decennie
ORDER BY decennie
""").show()

+--------+------------+
|decennie|total_matchs|
+--------+------------+
|    1870|          13|
|    1880|          55|
|    1890|          59|
|    1900|         124|
|    1910|         283|
|    1920|         729|
|    1930|        1010|
|    1940|         804|
|    1950|        1576|
|    1960|        2810|
|    1970|        3922|
|    1980|        4795|
|    1990|        6596|
|    2000|        9422|
|    2010|        9735|
|    2020|        2219|
+--------+------------+



In [44]:
spark.sql("""
SELECT equipe, SUM(defaite) as total_defaites
FROM (
    SELECT home_team as equipe, CASE WHEN COALESCE(TRY_CAST(home_score AS INT), 0) < COALESCE(TRY_CAST(away_score AS INT), 0) THEN 1 ELSE 0 END as defaite FROM football WHERE year(date) = 2022
    UNION ALL
    SELECT away_team as equipe, CASE WHEN COALESCE(TRY_CAST(away_score AS INT), 0) < COALESCE(TRY_CAST(home_score AS INT), 0) THEN 1 ELSE 0 END as defaite FROM football WHERE year(date) = 2022
)
GROUP BY equipe
HAVING total_defaites = 0
""").show()

+-------------+--------------+
|       equipe|total_defaites|
+-------------+--------------+
|      Bonaire|             0|
|    Argentina|             0|
|       Angola|             0|
|  Puerto Rico|             0|
|    Palestine|             0|
|      Georgia|             0|
|French Guiana|             0|
|      Namibia|             0|
|       Brazil|             0|
|  Netherlands|             0|
|    Maule Sur|             0|
+-------------+--------------+



In [47]:
spark.sql("""
WITH resultats AS (
    -- On normalise : chaque ligne est un match pour une équipe avec son résultat (V ou non)
    SELECT equipe, date,
           CASE WHEN (equipe = home_team AND COALESCE(TRY_CAST(home_score AS INT), 0) > COALESCE(TRY_CAST(away_score AS INT), 0)) OR
                     (equipe = away_team AND COALESCE(TRY_CAST(away_score AS INT), 0) > COALESCE(TRY_CAST(home_score AS INT), 0))
                THEN 1 ELSE 0 END as est_victoire
    FROM (SELECT home_team as equipe, date, home_team, away_team, home_score, away_score FROM football
          UNION ALL
          SELECT away_team as equipe, date, home_team, away_team, home_score, away_score FROM football)
),
groupes AS (
    -- Technique de différence de Row Numbers pour grouper les séries consécutives
    SELECT equipe, est_victoire,
           (ROW_NUMBER() OVER(PARTITION BY equipe ORDER BY date) -
            ROW_NUMBER() OVER(PARTITION BY equipe, est_victoire ORDER BY date)) as id_groupe
    FROM resultats
)
SELECT equipe, count(*) as serie_max
FROM groupes
WHERE est_victoire = 1
GROUP BY equipe, id_groupe
ORDER BY serie_max DESC
LIMIT 10
""").show()

+---------+---------+
|   equipe|serie_max|
+---------+---------+
|Mauritius|       17|
|  Padania|       15|
|    Spain|       15|
|   Brazil|       14|
|   Brazil|       14|
|   France|       14|
|Australia|       13|
|   Brazil|       13|
|   Guyana|       13|
|   Mexico|       13|
+---------+---------+



In [48]:
spark.sql("""
SELECT tournament, vainqueur, nb_victoires FROM (
    SELECT tournament, vainqueur, count(*) as nb_victoires,
           ROW_NUMBER() OVER(PARTITION BY tournament ORDER BY count(*) DESC) as rang
    FROM (
        SELECT tournament,
               CASE WHEN COALESCE(TRY_CAST(home_score AS INT), 0) > COALESCE(TRY_CAST(away_score AS INT), 0) THEN home_team
                    WHEN COALESCE(TRY_CAST(away_score AS INT), 0) > COALESCE(TRY_CAST(home_score AS INT), 0) THEN away_team
               END as vainqueur
        FROM football
    )
    WHERE vainqueur IS NOT NULL
    GROUP BY tournament, vainqueur
)
WHERE rang = 1
""").show()

+--------------------+------------------+------------+
|          tournament|         vainqueur|nb_victoires|
+--------------------+------------------+------------+
|     ABCS Tournament|          Suriname|           7|
|       AFC Asian Cup|              Iran|          41|
|AFC Asian Cup qua...|              Iran|          36|
|   AFC Challenge Cup|        Tajikistan|          11|
|AFC Challenge Cup...|       Philippines|           7|
|    AFF Championship|          Thailand|          42|
|AFF Championship ...|       Timor-Leste|           2|
|African Cup of Na...|             Egypt|          60|
|African Cup of Na...|       Ivory Coast|          67|
|African Nations C...|           Morocco|          11|
|African Nations C...|            Uganda|           4|
|    Afro-Asian Games|             India|           2|
|  Amílcar Cabral Cup|           Senegal|          38|
|            Arab Cup|              Iraq|          13|
|Arab Cup qualific...|           Lebanon|           3|
|        A

In [49]:
spark.sql("""
SELECT h.equipe,
       ROUND(v_dom / m_dom, 2) as ratio_victoire_domicile,
       ROUND(v_ext / m_ext, 2) as ratio_victoire_exterieur
FROM (
    SELECT home_team as equipe, count(*) as m_dom,
           SUM(CASE WHEN COALESCE(TRY_CAST(home_score AS INT), 0) > COALESCE(TRY_CAST(away_score AS INT), 0) THEN 1 ELSE 0 END) as v_dom
    FROM football GROUP BY home_team
) h
JOIN (
    SELECT away_team as equipe, count(*) as m_ext,
           SUM(CASE WHEN COALESCE(TRY_CAST(away_score AS INT), 0) > COALESCE(TRY_CAST(home_score AS INT), 0) THEN 1 ELSE 0 END) as v_ext
    FROM football GROUP BY away_team
) e ON h.equipe = e.equipe
ORDER BY ratio_victoire_domicile DESC
""").show(10)

+------------------+-----------------------+------------------------+
|            equipe|ratio_victoire_domicile|ratio_victoire_exterieur|
+------------------+-----------------------+------------------------+
| Western Australia|                    1.0|                    0.32|
|          Chameria|                    1.0|                    0.25|
|Parishes of Jersey|                    1.0|                     0.0|
|     Central Spain|                    1.0|                     0.0|
|          Cascadia|                    1.0|                     0.4|
|         Yorkshire|                    0.8|                     0.5|
|            Jersey|                   0.79|                     0.5|
|            Monaco|                   0.75|                    0.25|
|            Rhodes|                   0.73|                    0.43|
|           Menorca|                   0.73|                     0.2|
+------------------+-----------------------+------------------------+
only showing top 10 